In [1]:
import numpy as np
import pandas as pd 
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.
/usr/local/google/home/vere/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/google/home/vere/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/google/home/vere/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/google/ho

In [2]:
import os
print(os.listdir())

['IPL-Functional_LSTM-multiout.ipynb', 'machine-learning-ex7', 'IPL-Sequential-LSTM.ipynb', '.ipynb_checkpoints', 'org.octave.Octave.desktop', 'API requests.ipynb', 'machine-learning-ex6', 'machine-learning-ex1', 'machine-learning-ex8', 'deliveries.csv', 'machine-learning-ex4', 'machine-learning-ex2', 'IPL-Functional_LSTM.ipynb', 'machine-learning-ex5', 'matches.csv', 'machine-learning-ex3']


In [3]:
matches=pd.read_csv('matches.csv')
deliveries=pd.read_csv('deliveries.csv')
deliveries.player_dismissed=deliveries.player_dismissed.notnull().astype(int)
deliveries.columns


Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [4]:


# A= np.array(deliveries.loc[deliveries['match_id'] == 1])
# A.shape
deliveries['batting_team'].unique()
A = deliveries['batting_team'].str.get_dummies()
B = deliveries['bowling_team'].str.get_dummies()


In [5]:
deliveries = deliveries.join(A, lsuffix='_l', rsuffix='_r')
deliveries = deliveries.join(B, lsuffix='_batfirst', rsuffix='_bowlfirst')

deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,Kings XI Punjab_bowlfirst,Kochi Tuskers Kerala_bowlfirst,Kolkata Knight Riders_bowlfirst,Mumbai Indians_bowlfirst,Pune Warriors_bowlfirst,Rajasthan Royals_bowlfirst,Rising Pune Supergiant_bowlfirst,Rising Pune Supergiants_bowlfirst,Royal Challengers Bangalore_bowlfirst,Sunrisers Hyderabad_bowlfirst
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
deliveries.columns

feature_cols = ['match_id', 'inning','over', 'ball', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed', 'Chennai Super Kings_batfirst',
       'Deccan Chargers_batfirst', 'Delhi Capitals_batfirst',
       'Delhi Daredevils_batfirst', 'Gujarat Lions_batfirst',
       'Kings XI Punjab_batfirst', 'Kochi Tuskers Kerala_batfirst',
       'Kolkata Knight Riders_batfirst', 'Mumbai Indians_batfirst',
       'Pune Warriors_batfirst', 'Rajasthan Royals_batfirst',
       'Rising Pune Supergiant_batfirst', 'Rising Pune Supergiants_batfirst',
       'Royal Challengers Bangalore_batfirst', 'Sunrisers Hyderabad_batfirst',
       'Chennai Super Kings_bowlfirst', 'Deccan Chargers_bowlfirst',
       'Delhi Capitals_bowlfirst', 'Delhi Daredevils_bowlfirst',
       'Gujarat Lions_bowlfirst', 'Kings XI Punjab_bowlfirst',
       'Kochi Tuskers Kerala_bowlfirst', 'Kolkata Knight Riders_bowlfirst',
       'Mumbai Indians_bowlfirst', 'Pune Warriors_bowlfirst',
       'Rajasthan Royals_bowlfirst', 'Rising Pune Supergiant_bowlfirst',
       'Rising Pune Supergiants_bowlfirst',
       'Royal Challengers Bangalore_bowlfirst',
       'Sunrisers Hyderabad_bowlfirst']

In [7]:
n_a=64
feature_size = len(feature_cols)
# number of examples
m = len(deliveries.match_id.unique())
# List of all the match id's used in the dataset (note not in temporal order and some ids are quite random)
match_ids = deliveries.match_id.unique()

# sorted_devs[sorted_devs[:,0].argsort()]
Number_of_deliveries = [deliveries.match_id.value_counts(sort=False).to_dict()[x] for x in match_ids]

max_game_length = max(Number_of_deliveries)

# find the indices of the match starts in the original dataset
ind_start_of_match = np.cumsum(Number_of_deliveries)


# check =np.asarray(deliveries['ball'])[ind_start_of_match]
npdevs = np.asarray(deliveries[feature_cols])

# Set up NN layers with shareable weights
LSTM_cell = LSTM(n_a, return_state = True)  
reshapor = Reshape((1, feature_size)) 
densor = Dense(1,activation='sigmoid')

In [8]:
# Insert Nans so that each game is of equal length (the length of the max)
npdevs_pad = npdevs
insertion_counter =0
for ind,i  in enumerate(ind_start_of_match):
    balls_short_of_max = max_game_length-Number_of_deliveries[ind]
    a = np.zeros((balls_short_of_max,feature_size)) 
#     a[:] = np.nan
#     if ind==8:
#         print(Number_of_deliveries[ind])
#         print(a.shape)
    npdevs_pad = np.insert(npdevs_pad, i+insertion_counter, a, 0)
    insertion_counter += balls_short_of_max


In [9]:
# Find indices of the match starts in the dataset with Nan's added
ind_start_of_match_pad = np.cumsum(np.ones((ind_start_of_match.shape))*max_game_length)-max_game_length
np.asarray(npdevs_pad)[ind_start_of_match_pad.astype(int)[:-1]]
# ind_start_of_match_pad



array([[    1,     1,     1, ...,     0,     1,     0],
       [    2,     1,     1, ...,     0,     0,     0],
       [    3,     1,     1, ...,     0,     0,     0],
       ...,
       [11412,     1,     1, ...,     0,     0,     0],
       [11413,     1,     1, ...,     0,     0,     0],
       [11414,     1,     1, ...,     0,     0,     0]])

In [10]:
ind_start_of_match_pad=tf.convert_to_tensor(ind_start_of_match_pad.astype(int)[:-1])


In [11]:
max_game_length

267

In [12]:
# X shape is m by game length by feature size
def IPL_model(feature_size,feature_cols,n_a,max_game_length):
    
    X = Input(shape=(max_game_length,feature_size))
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    outputs = []
    
       # Loop over balls for the max game length
    for t in range(max_game_length):
        
        # select the "t"th time step vector from X. 
#         x = Lambda(lambda x: np.array(deliveries.loc[deliveries['match_id'] == t]))(X)
#         x = Lambda(lambda x: deliveries[deliveries[:,0] == t,:])(X)
#         x = Lambda(lambda x: x[ind_start_of_match_pad+t,:])(X)
#         x = Lambda(lambda x:   tf.gather(x,ind_start_of_match_pad+t))(X)
        x = Lambda(lambda x: x[:,t,:])(X)

        # Use reshapor to reshape x to be (1, feature_size) (≈1 line)
        x = reshapor(x)
        # Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x,initial_state=[a,c])
        # Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X,a0,c0],outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

In [13]:
model = IPL_model(feature_size,feature_cols,n_a,max_game_length)


In [14]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

# Don't care how much the team wins or loses by, 
# if the score in this col is greater than 1, then the bat first team won
Y_final = np.array(matches.win_by_runs.clip(upper=1))
Y = Y_final.reshape(1,m)*np.ones((max_game_length,m))
X = np.transpose(npdevs_pad.reshape((max_game_length,m,feature_size),order='F' ),(1, 0, 2))
# X shape is m by game length by feature size
Y.shape

(267, 756)

In [ ]:
model.fit([X, a0, c0], list(Y), verbose = 1, epochs=200, validation_split=0.1,shuffle=True)

In [17]:

# score, acc = model.evaluate(x_test, y_test)